In [20]:
import numpy as np
import pandas as pd
import math
from sklearn.base import BaseEstimator, ClassifierMixin
class DecisionTreeC45Node(BaseEstimator, ClassifierMixin):
    def __init__(self, max_depth, cat_features=None):
        # property
        self.max_depth = max_depth

        # constant
        self.cat_features = cat_features
        # variables
        self.children = list()
        self.isleaf = False

        self.split_val = None
        self.split_feature = None
        self.is_cat_feature = False
        self.classification = None

        self.depth = 0

    def is_pure(self, y):
        # check whether the branch is pure() having same class )

        # compare all class label with the class of first row.
        #         print(y)
        for i in y:
            if i != y[0]:
                return False
        return True

    def find_split_val(self, X, y, potential_features):
        # Find best split value

        class_values = list(set(y))     
        b_feature, b_value, b_score, b_groups = None, None, None, None
        
        entropy_parent = self.entropy([y], class_values)
        for feature in potential_features:
            if feature in self.cat_features:
                groups = self.split(feature, None, X, y)
                groups_y = list()
                split_values = list()
                for v,gr in groups.items():
                    groups_y.append(gr['y'])
                    split_values.append(v)
                entropy_children = self.entropy(groups_y, class_values)
                split_info = self.splitting_information(groups_y)
                if split_info == 0:
                    gain_ratio = 0
                else:
                    gain_ratio = (entropy_parent - entropy_children)
#                 print('X{:d} ={} E={:.3f},IG={:.3f},SI={:.3f},GR={:.3f}'.format((feature + 1), 
#                                                                                 split_values, 
#                                                                                 entropy_children, 
#                                                                                 (entropy_parent - entropy_children), 
#                                                                                 split_info, gain_ratio))
                if b_score is None or gain_ratio > b_score:
                    b_index, b_value, b_score, b_groups, is_cat_feature = feature, split_values, gain_ratio, groups, True
            else:
                for row in X:
                    groups = self.split(feature, row[feature], X, y)
                    groups_y = list()
                    for _,gr in groups.items():
                        groups_y.append(gr['y'])
                    entropy_children = self.entropy(groups_y, class_values)
                    split_info = self.splitting_information(groups_y)
                    if split_info == 0:
                        gain_ratio = 0
                    else:
                        gain_ratio = (entropy_parent - entropy_children)
#                     print('X{:d} ={} E={:.3f},IG={:.3f},SI={:.3f},GR={:.3f}'.format((feature + 1), row[feature], entropy_children, (entropy_parent - entropy_children), split_info, gain_ratio))
                    if b_score is None or gain_ratio > b_score:
                        b_index, b_value, b_score, b_groups, is_cat_feature = feature, row[feature], gain_ratio, groups, False
        return {'index': b_index, 'value': b_value, 'groups': b_groups, 'gain_ratio': b_score, 'is_cat_feature': is_cat_feature}

    def split(self, feature, val, X, y):
        # split data according to split criteria
        groups = dict()
        if feature in self.cat_features:
            # categorical feature ( multiple branches )
            # create branches for each category
            for idx, row in enumerate(X):
                if row[feature] in groups.keys():
                    groups[row[feature]]['X'].append(row)
                    groups[row[feature]]['y'].append(y[idx])
                else:
                    groups[row[feature]] = {'X': [row], 'y': [y[idx]]}
        else:
            # numerical feature (binary branches)
            # data with feature value smaller than val goes to left ( feature < val )
            # the rest goes to right
            groups['left'] = {'X':list(), 'y':list()}
            groups['right'] = {'X':list(), 'y':list()}
            for idx, row in enumerate(X):
                if row[feature] < val:
                    groups['left']['X'].append(row)
                    groups['left']['y'].append(y[idx])
                else:
                    groups['right']['X'].append(row)
                    groups['right']['y'].append(y[idx])

        return groups

    def entropy(self, groups, classes):
        # calculte entropy of a split
        n_instances = 0
        for gr in groups:
            n_instances += len(gr)
        
        entropy = 0.0
        for gr in groups:
            size = len(gr)
            # avoid divide by zero
            if size == 0:
                continue
            score = 0.0

            # score the group based on the score for each class
            for class_val in classes:
                p = 0.0
                for v in gr:
                    if v == class_val:
                        p += 1
                p = p / size
                if p > 0:
                    score += p * math.log2(p)
            # weight the group score by its relative size
            entropy += (score) * (size / n_instances)  # *0.5
        entropy = -entropy
        return entropy

    def splitting_information(self, groups):
        n_instances = 0
        for gr in groups:
            n_instances += len(gr)
#             print(len(gr))
        
        splitting_info = 0.0
        for gr in groups:
            t = len(gr)/n_instances
            if t > 0.0:
                splitting_info += t * np.log2(t)
        splitting_info = -splitting_info
        return splitting_info

    def grow(self, X, y, depth, potential_features):
#         print("\ndepth={}, p_features={}".format(depth,potential_features))
        if self.is_pure(y) or self.max_depth <= depth or len(potential_features) == 0:
#             print("terminate at depth={}".format(depth))
            self.terminate(y, depth)
            return
        else:

            best_split = self.find_split_val(X, y, potential_features)
            self.split_val = best_split['value']
            self.split_feature = best_split['index']
            self.is_cat_feature = best_split['is_cat_feature']
            print("split on feature:{} GR={}".format(self.split_feature, best_split['gain_ratio']))
            
            if self.is_cat_feature:
                potential_features.remove(self.split_feature)
            for _, gr in best_split['groups'].items():
                node = DecisionTreeC45Node(self.max_depth, self.cat_features)
                node.grow(gr['X'], gr['y'], depth + 1, potential_features)
                self.children.append(node)            
            #             print("{}X{} < {} gini={}".format(self.depth*' ',self.split_feature+1, self.split_val, best_split['gini']))
            #             print("left - Class 0:{},class 1:{}".format(np.sum(left.iloc[:,-1]==0),np.sum(left.iloc[:,-1]==1)))
            #             print("right - Class 0:{},class 1:{}".format(np.sum(right.iloc[:,-1]==0),np.sum(right.iloc[:,-1]==1)))

        self.depth = depth
        return

    def terminate(self, y, depth):
        # define leaf node
        # most frequent class in the data as class label of this node
        self.classification = max(set(y), key=y.count)
        self.isleaf = True
        self.depth = depth

    def fit(self, X, y):
        # grow a tree
        n = len(X.columns)
        
        # convert cat_feature to its index in data X's columns
        if self.cat_features is None:
            self.cat_features = []
        else:
            i = 0
            cat_features_idx = []
            for col in X.columns:
                if col in self.cat_features:
                    cat_features_idx.append(i)
                i += 1
            self.cat_features = cat_features_idx
        
        X = X.values.tolist()
        y = y.values.tolist()
        potential_features = np.linspace(0,n-1,n,dtype=np.int32).tolist()
        self.grow(X, y, 1, potential_features)
        return self

    def print_tree(self):
        if not self.isleaf:
            if self.is_cat_feature:
                for i in range(len(self.split_val)):
                    print("{}X{} = {} ".format(self.depth * ' ', self.split_feature, self.split_val[i]))
                    self.children[i].print_tree()
            else:
                print("{}X{} < {} ".format(self.depth * ' ', self.split_feature, self.split_val))
                self.children[0].print_tree()
                self.children[1].print_tree()
        else:
            print("{}[{}]".format(self.depth * ' ', self.classification))

    def predict_iterate(self, row):

        if self.isleaf:
            # is leaf node
            return self.classification
        else:
            # not leaf node
            if self.is_cat_feature:
                # predict categorical feature
                for i in range(len(self.split_val)):
                    if row[self.split_feature] == self.split_val[i]:
                        return self.children[i].predict_iterate(row)
                return None
            else:
                # predict numerical feature
                if row[self.split_feature] < self.split_val:
                    return self.children[0].predict_iterate(row)
                else:
                    return self.children[1].predict_iterate(row)

    def predict(self, X):
        num_rows = X.shape[0]
        prediction = []
        #         prediction = np.zeros((num_rows,1))
        for idx, row in X.iterrows():
            prediction.append(self.predict_iterate(row))

        return np.array(prediction)

    def prune(self, tree):
        # prune here
        pruned_tree = tree
        return pruned_tree


In [21]:
import pandas as pd

data = pd.DataFrame([[23.771244718,1.784783929, 'a',0],
[11.728571309,55.169761413, 'a',0],
[3.678319846,3.81281357, 'a',0],
[3.961043357,0.61995032, 'a',0],
[2.999208922,2.209014212, 'b',0],
[7.497545867,3.162953546, 'b',1],
[9.00220326,3.339047188, 'b',1],
[7.444542326,0.476683375, 'b',1],
[10.12493903,3.234550982, 'b',1],
[6.642287351,3.319983761, 'b',1]])
# data.columns = ['1','2','3','class']
# data

In [22]:
dt = DecisionTreeC45Node(cat_features=[2], max_depth=10)
dt.fit(X=data.iloc[:,:-1],y=data.iloc[:,-1])

split on feature:2 GR=0.6099865470109875
split on feature:0 GR=0.6500224216483541


DecisionTreeC45Node(cat_features=[2], max_depth=10)

In [23]:
dt.predict(X=data.iloc[:,:-1])

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [24]:
dt.print_tree()

 X2 = a 
  [0]
 X2 = b 
  X0 < 6.642287351 
   [0]
   [1]


# Test on CKD dataset

In [25]:
df = pd.read_csv("./dataset/chronic_kidney_disease_full.csv", na_values=['?','\t?'])
print(df.shape)
df = df.drop(columns = ['id'])
df.head()

(400, 26)


,'age','bp','sg','al','su','rbc','pc','pcc','ba','bgr',...,'pcv','wbcc','rbcc','htn','dm','cad','appet','pe','ane','class'
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


In [26]:
# Fill NA values with most frequent values in the column
df = df.fillna(df.mode().iloc[0])
df.isnull().sum().sum()

0

In [27]:
new_col = []
for col in df.columns:
    col = col.replace("'",'')
    new_col.append(col)
df.columns = new_col

str_cols = []
str_cols_index = []
num_cols = []
i = 0
for col in df.drop(columns = 'class').columns:
    if df[col].dtype != np.int64 and df[col].dtype != np.float64:
        str_cols.append(col)
        str_cols_index.append(i)
    else:
        num_cols.append(col)
    i += 1
    
print("categorical columns: {}\n".format(str_cols))
print("categorical column index: {}\n".format(str_cols_index))
print("numerical columns: {}\n".format(num_cols))

categorical columns: ['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane']

categorical column index: [5, 6, 7, 8, 18, 19, 20, 21, 22, 23]

numerical columns: ['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc']



In [28]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['class'])
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)


In [29]:
dt = DecisionTreeC45Node(cat_features=str_cols, max_depth=5)
# %prun dt.train(X=X_train,y=y_train, cat_features=str_cols, max_depth=5)
dt.fit(X=X_train,y=y_train)

split on feature:11 GR=0.5099416058315251
split on feature:2 GR=0.5863755004101161
split on feature:14 GR=0.2145075136979618
split on feature:1 GR=0.05943853804639755


DecisionTreeC45Node(cat_features=[5, 6, 7, 8, 18, 19, 20, 21, 22, 23],
          max_depth=5)

In [30]:
pred = dt.predict(X_test)

print(np.sum(pred == y_test)/y_test.shape[0])

0.9772727272727273


In [31]:
dt.print_tree()

 X11 < 1.3 
  X2 < 1.02 
   [ckd]
   X14 < 13.0 
    [ckd]
    X1 < 90.0 
     [notckd]
     [ckd]
  [ckd]


In [32]:
dt.cat_features

[5, 6, 7, 8, 18, 19, 20, 21, 22, 23]